In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from analytics_tools.analytics_tools3 import remove_extrema
%matplotlib inline

In [2]:
df = pd.read_csv('Datos/Sales_Transactions_Dataset_Weekly.csv')

In [3]:
df = df[['Product_Code']+[v for v in df.columns if v[:1]=='W']]

In [4]:
df.rename(columns={'Product_Code':'id'},inplace=True)

In [5]:
df.head()

,id,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9


In [6]:
%%time
lst_df = []
for t in range(11,48):
    var = ['W%d'%i for i in range(t-12+1,t+1)]
    var = list(reversed(var))
    aux = df[['id']+var].copy()
    aux.rename(columns=dict(zip(var,['x%d'%i for i in range(1,12+1)])),inplace=True)
    aux['ancla'] = t
    var = ['W%d'%i for i in range(t+1,t+4+1)]
    tgt = df[['id']+var].copy()
    tgt.rename(columns=dict(zip(var,['y%d'%i for i in range(1,4+1)])),inplace=True)
    aux = aux.merge(tgt,on='id',how='inner')
    lst_df.append(aux)
df = pd.concat(lst_df,ignore_index=True)

CPU times: user 227 ms, sys: 4.16 ms, total: 231 ms
Wall time: 234 ms


In [7]:
df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,ancla,y1,y2,y3,y4
0,P1,14,11,14,6,21,14,12,13,8,10,12,11,11,16,9,9,9
1,P2,2,2,3,3,3,6,1,7,2,3,6,7,11,6,2,0,6
2,P3,9,14,6,12,13,7,8,10,9,8,11,7,11,4,7,12,8
3,P4,4,8,11,13,13,9,6,9,5,13,8,12,11,5,4,15,7
4,P5,18,11,9,9,14,9,7,6,11,13,5,8,11,8,4,13,8


In [8]:
def ninc(x):
    return sum([int(a>b) for a,b in zip(x,x[1:])])
def delta(x):
    return np.mean([(a-b) for a,b in zip(x,x[1:])])

In [ ]:
lst_func = ['mean','median','max','min','std',ninc,delta]

In [ ]:
%%time
for k in range(2,14,2):
    for f in lst_func:
        if type(f) == str:
            nombre = f
        else:
            nombre = f.func_name
        df['v_%s_%d'%(nombre,k)] = df[['x%d'%i for i in range(1,k+1)]].apply(f,axis=1)

In [ ]:
df.head()

In [ ]:
df_noext = remove_extrema(df=df,
               features=[v for v in df.columns if v[:2]=='v_'],
               lower_bound=1,
               upper_bound=99,
               removed_threshold=0.1)

In [ ]:
X = df_noext[[v for v in df.columns if v[:2]=='v_']].copy()
y = df_noext[['y1']].copy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scx = MinMaxScaler()
scy = MinMaxScaler()
scx.fit(X)
scy.fit(y)
Xs = pd.DataFrame(scx.transform(X),columns=X.columns)
ys = pd.DataFrame(scy.transform(y))

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
Xt,Xv,yt,yv = train_test_split(Xs,ys,train_size=0.7)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
help(ExtraTreesRegressor)

In [ ]:
param_grid = dict(n_estimators=range(100,200,10),
                  criterion = ['mse','mae'],
                  max_features = range(20,30),
                  max_depth = range(10,20)
                 )

In [ ]:
model = ExtraTreesRegressor()
model.fit(Xt,yt)

print r2_score(y_pred=model.predict(Xt),y_true=yt)
print r2_score(y_pred=model.predict(Xv),y_true=yv)

In [ ]:
grid = RandomizedSearchCV(cv=3,verbose=True,
                          estimator=model,
                          n_jobs=-1,
                          param_distributions=param_grid,
                          scoring='r2')

In [ ]:
grid.fit(Xs,ys)
model = grid.best_estimator_

In [ ]:
print r2_score(y_pred=model.predict(Xt),y_true=yt)
print r2_score(y_pred=model.predict(Xv),y_true=yv)

In [ ]:
df_noext['y^'] = scy.inverse_transform(model.predict(scx.transform(X)))

In [ ]:
aux = df_noext.loc[df.id=='P1'][['y1','y^','id','ancla']]

In [ ]:
plt.plot(aux['ancla'],aux['y1'])
plt.plot(aux['ancla'],aux['y^'])